# Recommender System using Collaborative Filtering


• Input 

    • User-Rating Matrix (Incomplete : Sparse)

• Output

    • For a particular user, complete the row
 
<br>

User Based Collaborative Filtering:<br>
        
       • If user-u likes item-j, recommend item-j’ that was liked by other users like him

<img src="img/UBCF.png">

<br><br><br>
Item Based Collaborative Filtering:<br>


    • If user-u likes item-j, recommend item-j’ that is similar to item-j
    

<img src="img/IBCF.png">

### Surprise is a python package used to make recommender systems

Surprise is an easy-to-use Python scikit for recommender systems.

There are some frequently used modules in Suprise package<br>


<li>similarities module</li>
    
    The similarities module includes tools to compute similarity metrics between users or items.
    
    Popular ones are as shown below
    
    cosine --- Compute the cosine similarity between all pairs of users (or items).
    msd -- Compute the Mean Squared Difference similarity between all pairs of users (or items).
    pearson -- Compute the Pearson correlation coefficient between all pairs of users (or items).

<li>accuracy module</li>

    The accuracy module provides with tools for computing accuracy metrics on a set of predictions.
    
    Popular choices are as shown below
    
    rmse -- Compute RMSE (Root Mean Squared Error).
    mae -- Compute MAE (Mean Absolute Error).

<li> dataset module</li>

    The dataset module defines the Dataset class and other subclasses which are used for managing datasets.
    
    Popular choices are shown below
    
    Dataset.load_builtin -- Load a built-in dataset.
    Dataset.load_from_file -- Load a dataset from a (custom) file.
    Dataset.load_from_folds -- Load a dataset where folds (for cross-validation) are predefined by some files.
    Dataset.load_from_df -- Load a dataset from a pandas dataframe.
    
<li>Reader Module</li>

    The Reader class is used to parse a file containing ratings. Such a file is assumed to specify only one 
    rating  per line, and each line needs to respect the following structure:
    
    user ; item ; rating ; [timestamp]
    
    where the order of the fields and the separator (here ‘;’) may be arbitrarily defined. 
    brackets indicate that the timestamp field is optional.


http://surpriselib.com/
<br>https://github.com/NicolasHug/Surprise
<br>http://surprise.readthedocs.io/en/stable/index.html

In [1]:
#### Install surprise package using Anaconda
#! conda install -c anaconda cython
#!conda install -c conda-forge scikit-surprise

In [2]:
from surprise import Dataset
from surprise import Reader, KNNWithMeans
from surprise.model_selection import cross_validate
from surprise import accuracy
import pandas as pd

#### Jokes Dataset
http://eigentaste.berkeley.edu/dataset/
<br>149 Jokes
<br>59132 Users
<br>Reading jokes files
<br>Note - Data is tab seperated

In [3]:
jokes = pd.read_csv("input/jester_items.tsv",sep="\t",names=["ItemID","Joke"])

In [4]:
jokes.shape

(149, 2)

In [5]:
jokes.head()

,ItemID,Joke
0,1:,"A man visits the doctor. The doctor says, ""I h..."
1,2:,This couple had an excellent relationship goin...
2,3:,Q. What's 200 feet long and has 4 teeth? A. Th...
3,4:,Q. What's the difference between a man and a t...
4,5:,Q. What's O. J. Simpson's web address? A. Slas...


#### Reading the ratings file

In [6]:
ratings = pd.read_csv("input/jester_ratings.csv", index_col=None)
ratings.head()

,UserID,ItemID,Rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875


In [7]:
ratings.shape

(1761439, 3)

#### Check the unique users and unique jokes that were rated

In [8]:
ratings.UserID.nunique()

59132

In [9]:
ratings.ItemID.nunique()

140

#### Get the summary of the dataset

In [10]:
#Observe the ratings 
ratings.describe()

,UserID,ItemID,Rating
count,1.761439e+06,1.761439e+06,1.761439e+06
mean,3.272322e+04,7.071133e+01,1.618602e+00
std,1.828011e+04,4.600790e+01,5.302608e+00
min,1.000000e+00,5.000000e+00,-1.000000e+01
25%,1.720200e+04,2.100000e+01,-2.031000e+00
50%,3.480800e+04,6.900000e+01,2.219000e+00
75%,4.730600e+04,1.120000e+02,5.719000e+00
max,6.397800e+04,1.500000e+02,1.000000e+01


### Defining the parser to read data into surprise dateframe
#### The parser requires the scale of ratings, and the columns to be mentioned using rating_scale and line_format

Lets limit to 1000 users for sake of convenience 

In [11]:
no_of_users = 1000
reader = Reader(line_format = 'user item rating', rating_scale=(-10, 10))
data = Dataset.load_from_df(ratings[ratings.UserID < no_of_users], reader)

In [12]:
data

#### Simulation Parameters
-  Algorithm Type
-  User-Based vs Item-Based
-  Similarity Metric

In [13]:
parameters = {'name': 'cosine', 'user_based': True}
model = KNNWithMeans(k=5,sim_options=parameters)

#### Cross Validation Accuracies

In [14]:
cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...


/Users/bhaskarn/.virtualenvs/env_datascience/lib/python3.7/site-packages/surprise/prediction_algorithms/algo_base.py:248: RuntimeWarning: invalid value encountered in double_scalars
  sim = construction_func[name](*args)


Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    5.1232  5.1192  5.0801  5.0288  5.0791  5.0861  0.0342  
MAE (testset)     3.9390  3.9437  3.9095  3.8763  3.9182  3.9173  0.0241  
Fit time          0.65    0.64    0.65    0.64    0.64    0.64    0.00    
Test time         1.21    1.17    1.12    1.13    1.14    1.15    0.03    


{'test_rmse': array([5.12321477, 5.11917683, 5.08005101, 5.02879195, 5.07906867]),
 'test_mae': array([3.93895661, 3.94369676, 3.90950575, 3.87628569, 3.91824324]),
 'fit_time': (0.645963191986084,
  0.6423938274383545,
  0.6482410430908203,
  0.643779993057251,
  0.6419210433959961),
 'test_time': (1.2149181365966797,
  1.1664659976959229,
  1.1178488731384277,
  1.1332950592041016,
  1.1355020999908447)}

#### Training the model on complete data

In [15]:
trainset = data.build_full_trainset()
print(trainset)

In [16]:
# Train the algorithm on the trainset
model.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [17]:
# Then predict ratings for all pairs (uid, iid) that are NOT in the training set.
testset = trainset.build_anti_testset(fill=0)
# print(testset)

In [18]:
predictions = model.test(testset)

In [19]:
# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 4.3255


4.325504919373649

#### Filtering instances which can be used for predictions

In [20]:
predictions[0:2]

[Prediction(uid=1, iid=28, r_ui=0.0, est=1.7192563561018042, details={'actual_k': 5, 'was_impossible': False}),
 Prediction(uid=1, iid=30, r_ui=0.0, est=0.3186739307458346, details={'actual_k': 5, 'was_impossible': False})]

#### Function to calculate top 10 predictions for each user

In [21]:
from collections import defaultdict

In [22]:
top_n = defaultdict(list)

In [23]:
top_n

defaultdict(list, {})

In [24]:
for uid, iid, true_r, est, _ in predictions:
    top_n[uid].append((iid, est))

In [25]:
len(top_n[3])

122

In [26]:
n = 5

# Then sort the predictions for each user and retrieve the k highest ones.
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:n]

In [27]:
top_n[1]

[(114, 6.810007574102432),
 (145, 6.791053391474415),
 (147, 6.615728148102168),
 (122, 5.94905167179692),
 (130, 5.700379243011843)]

In [28]:
# Fetching top 10 predictions for each user
from collections import defaultdict

def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

top_n = get_top_n(predictions, n=10)
take(5, top_n.items())

[(1,
  [(114, 6.810007574102432),
   (145, 6.791053391474415),
   (147, 6.615728148102168),
   (122, 5.94905167179692),
   (130, 5.700379243011843),
   (138, 5.674433396720778),
   (56, 5.591811874055171),
   (137, 5.4133909973537),
   (129, 5.233686593332871),
   (139, 5.230990241687502)]),
 (2,
  [(121, 7.416008020576454),
   (105, 7.282764912379616),
   (126, 6.808174935869103),
   (114, 6.787986030334784),
   (106, 6.77074078289669),
   (111, 6.313558626485987),
   (130, 6.234922241837243),
   (129, 6.199921588916162),
   (108, 6.096442494649493),
   (56, 5.924515133346259)]),
 (3,
  [(139, -0.5635992502265763),
   (138, -0.7066441133757939),
   (76, -1.9087347922674418),
   (143, -2.0165559011011025),
   (66, -2.43453905662808),
   (114, -2.467397097069979),
   (110, -2.593194883392779),
   (106, -2.673850454289787),
   (36, -2.9587137353268176),
   (35, -3.1323642369559934)]),
 (4,
  [(145, 2.706551686726784),
   (17, 1.7750763951140787),
   (116, -0.4865097703309109),
   (26, -0

#### Top Predictions Matrix

In [29]:
# Printing top predictions
for uid, user_ratings in take(10,top_n.items()):
    print(uid, [iid for (iid, _) in user_ratings])

1 [114, 145, 147, 122, 130, 138, 56, 137, 129, 139]
2 [121, 105, 126, 114, 106, 111, 130, 129, 108, 56]
3 [139, 138, 76, 143, 66, 114, 110, 106, 36, 35]
4 [145, 17, 116, 26, 77, 130, 35, 119, 92, 40]
5 [117, 134, 114, 129, 150, 148, 128, 131, 115, 110]
6 [135, 127, 76, 102, 132, 113, 119, 107, 125, 120]
7 [77, 127, 105, 148, 28, 38, 118, 117, 122, 47]
8 [143, 132, 134, 147, 142, 145, 114, 128, 138, 136]
9 [116, 68, 104, 27, 141, 40, 66, 117, 60, 53]
10 [114, 138, 148, 122, 140, 111, 127, 120, 113, 126]


#### Top Jokes for each User

In [30]:
# Printing top predictions
for uid, user_ratings in take(10,top_n.items()):
    print("For User",uid)
    for  (iid, _) in user_ratings:
        print(iid)
        print(jokes.loc[int(iid)-1,"Joke"]) #iid -1 as row index in dataframe starts from 0 and not 1

For User 1
114
Sherlock Holmes and Dr. Watson go on a camping trip, set up their tent, and fall asleep. Some hours later, Holmes wakes his faithful friend. "Watson, look up at the sky and tell me what you see." Watson replies, "I see millions of stars." "What does that tell you?" Watson ponders for a minute. "Astronomically speaking, it tells me that there are millions of galaxies and potentially billions of planets. Astrologically, it tells me that Saturn is in Leo. Timewise, it appears to be approximately a quarter past three. Theologically, it's evident the Lord is all-powerful and we are small and insignificant. Meteorologically, it seems we will have a beautiful day tomorrow. What does it tell you?" Holmes is silent for a moment, then speaks. "Watson, you idiot, someone has stolen our tent."
145
A blonde, brunette, and a red head are all lined up to be shot to death by a firing squad. The brunette shouts, "Tornado!" and the riflemen turn around to see the tornado. It isn't there, 

KeyError: 149